In [12]:
from sklearn.model_selection import KFold
import numpy as np
import json


# Sample data
#2454 = 3 * 818
#split 818 scenes, then reaugment
scene_count = 818
n_augment = 3

pre_data = np.array(range(scene_count))

# Prepare cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=1)


splits = {}

kfold_split = kfold.split(pre_data)

for fold, (pre_train, pre_test) in enumerate(kfold_split, 1):
    test = [range(n_augment * i, n_augment * (i+1)) for i in pre_test]
    test = [j for sub in test for j in sub]
    train = [range(n_augment * i, n_augment * (i+1)) for i in pre_train]
    train = [j for sub in train for j in sub]

    splits[fold] = {'train':train, 'test':test}
    for i in range(2454):
        if i in test and i in train:
            print("ERROR: Overlap detected")


In [17]:
import os
import sys
sys.path.append(os.getcwd() + '/../')


from main_nusc import data_path
with open(data_path + '/../splits.json', 'w') as f:
    json.dump(splits, f)

: 

In [1]:
import sys
import os
sys.path.append(os.getcwd() + '/../')

from dataset_utils import *
from main_nusc import data_path
import pandas as pd
from ast import literal_eval

In [3]:
nusc_data = NuScenesSimple(data_path, [str(i) for i in range(2454)], mode='test', obs_perc=1)

lane_change_idxs = []
for i,n in enumerate(nusc_data):
    action_list = n['actions'].int().tolist() 
    if 8 in action_list or 9 in action_list:
        lane_change_idxs.append(i)

print(len(lane_change_idxs)/3)

In [ ]:
split_dict = {'train':lane_change_idxs, 'test':lane_change_idxs}
with open('../custom_splits/lane_changes.json', 'w') as f:
    json.dump(split_dict, f)